In [1]:
BASE_DIR = '../'*3
RUN_DIR = BASE_DIR + 'code/RNN/rnn_ext/rnn_ext_model.ipynb'

In [2]:
%run {RUN_DIR}

In [3]:
class Autoencoder(RnnExtModel):
    def __init__(self, name, dataset, hconfigs, show_maps = False, l2_decay = 0, \
                l1_decay = 0, dump_structure = False, fix_encoder = False):
        self.fix_encoder = fix_encoder
        # encoder학습 동결시킬지 (파라미터 가변 or 고정)
        super(Autoencoder, self).__init__(name, dataset, hconfigs,\
                                         show_maps, l2_decay, l1_decay, dump_structure)
        

In [ ]:
def autoencoder_init_parameters(self, hconfigs):
    econf = hconfigs['encoder']
    dconf = hconfigs['decoder']
    hconf = hconfigs['supervised']
    
    in_shape = self.dataset.input_shape
    
    pme, code_shape = self.build_subnet(econf, in_shape)
    pmd, represent_shape = self.build_subnet(dconf, code_shape)
    pmh, hidden_shape = self.build_subnet(hconf, code_shape)
    
    self.econfigs, self.dconfigs, self.hconfigs = econf, dconf, hconf
    
    self.pm_encoder, self.pm_decoder, self.pm_hiddens = pme, pmd, pmh
    
    
    output_cnt = int(np.prod(self.dataset.output_shape))
    self.seqout=False
    
    if len(hconf) > 0 and get_layer_type(hconf[-1]) in ['rnn','lstm']:
        if get_conf_param(hconf[-1], 'outseq',True):
            self.seqout = True
            hidden_shape = hidden_shape[1:]
            output_cnt = int(np.prod(self.dataset.output_shape[1:]))
            
    self.pm_output, _ = self.alloc_layer_param(hidden_shape, output_cnt)
    
    
def autoencoder_build_subnet(self, hconfigs, prev_shape):
    pms = []
    
    for hconfig in hconfigs:
        
        pm, prev_shape = self.alloc_layer_param(prev_shape, hconfig)
        pms.append(pm)
        
    return pms, prev_shape


Autoencoder.build_subnet = autoencoder_build_subnet
Autoencoder.init_parameters = autoencoder_init_parameters

In [ ]:
def autoencoder_autoencode(self,epoch_count = 10, batch_size=10,\
                           learning_rate=0.001, report=0):
    self.learning_rate = learning_rate
    batch_count = self.dataset.autoencode_count // batch_size
    time1 = time2 = int(time.time())

    model, optim, loss = self.Auto_get_torch_params()
    
    if report !=0:
        print('Model {}autoencode started:'.format(self.name))
        
    for epoch in range(epoch_count):
        costs =[]
        accs =[]
        self.dataset.shuffle_train_data(batch_size * batch_count)
        for n in range(batch_count):
            trX = self.dataset.get_autoencode_data(batch_size, n)                        
            cost, acc = self.autoencode_torch_step(trX, model, optim, loss)
#             cost, acc = self.autoencode_step(trX) # 
            
            costs.append(cost)
            accs.append(acc)
            
        if report > 0 and(epoch +1) % report == 0:
            acc = np.mean(accs)
            time3=int(time.time())
            tm1, tm2 = time3 - time2, time3 - time1
            self.dataset.train_prt_result(epoch+1, costs, accs, acc, tm1, tm2)
            time2 = time3
            
    tm_total = int(time.time()) - time1
    if report != 0:
        print('Model {} autoencode ended in {} secs :'.format(self.name, tm_total))
        
Autoencoder.autoencode = autoencoder_autoencode

In [83]:
class Autoencoder_layers(nn.Module, Autoencoder):
    def __init__(self):
        super().__init__()
        self.pm_encoder = nn.Linear(2,2)
        self.pm_decoder = nn.Linear(2,2)
        self.encoder_layers = nn.Sequential(self.pm_encoder)
        self.decoder_layers = nn.Sequential(self.pm_decoder)
        
    
    def forward(self, x):
        
        self.encoder_forward = self.encoder_layers(x)
        self.decoder_forward = self.decoder_layers(encoder)
        
        return self.decoder_forward

In [ ]:
def Auto_get_torch_params(self):
    
    model = Autoencoder_layers()
    optim = torch.optim.Adam(model.params, self.learning_rate)
    loss = nn.MSELoss()
    
    return model, optim, loss

Autoencoder.Auto_get_torch_params=Auto_get_torch_params

In [ ]:
def autoencode_torch_step(self, trX, model, optim, loss):
    optim.zero_grad()
    
    output=model.forward(trX)
    
    cost=loss(trX, output)
    
    acc = 1-(torch.sqrt(loss)/torch.mean(trX))

    loss.backward()
    
    optim.step()
    
    return cost, acc

In [ ]:
#지도 학습 
def autoencoder_forward_neuralnet(self, x ):
    hidden = x
    
    aux_encoder =[]
    
    for n, hconfig in enumerate(self.econfigs):
        hidden,aux = self.forward_layer(hidden, hconfig, self.pm_encoder[n])
        aux_encoder.append(aux)
        
    output, aux_layers = super(Autoencoder, self).forward_neuralnet(hidden)
    
    return output, [aux_encoder, aux_layers]


Autoencoder.forward_neuralnet = autoencoder_forward_neuralnet